In [12]:
import tensorflow as tf

2025-02-19 12:04:06.638121: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [13]:
x = tf.range(4, dtype=tf.float32)
x

<tf.Tensor: shape=(4,), dtype=float32, numpy=array([0., 1., 2., 3.], dtype=float32)>

In [15]:
x = tf.Variable(x)
x

<tf.Variable 'Variable:0' shape=(4,) dtype=float32, numpy=array([0., 1., 2., 3.], dtype=float32)>

In [20]:
# Record all computations onto a tape
with tf.GradientTape() as t:
    y = 2 * tf.tensordot(x, x, axes=1)
y

<tf.Tensor: shape=(), dtype=float32, numpy=28.0>

In [21]:
x_grad = t.gradient(y, x)
x_grad

<tf.Tensor: shape=(4,), dtype=float32, numpy=array([ 0.,  4.,  8., 12.], dtype=float32)>

In [22]:
with tf.GradientTape() as t:
    y = tf.reduce_sum(x)
t.gradient(y, x)  # Overwritten by the newly calculated gradient

<tf.Tensor: shape=(4,), dtype=float32, numpy=array([1., 1., 1., 1.], dtype=float32)>

In [23]:
tf.reduce_sum(x)

<tf.Tensor: shape=(), dtype=float32, numpy=6.0>

In [24]:
tf.tensordot(x, x, axes=1)

<tf.Tensor: shape=(), dtype=float32, numpy=14.0>

In [28]:
%%markdown
## Following code is taken from tensorflow documentation: https://www.tensorflow.org/guide/autodiff

## Following code is taken from tensorflow documentation: https://www.tensorflow.org/guide/autodiff


In [29]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf